In [ ]:
%matplotlib inline

In [ ]:
import uproot
import matplotlib.pyplot as plt

import vector
import awkward
import numpy as np

In [ ]:
def map_pdgid_to_candid(pdgid, charge):
    if pdgid == 0:
        return 0

    #photon, electron, muon
    if pdgid in [22, 11, 13, 15]:
        return pdgid

    # charged hadron
    if abs(charge) > 0:
        return 211

    # neutral hadron
    return 130

In [ ]:
fi = uproot.open("/home/joosep/particleflow/fcc/p8_ee_Z_Ztautau_ecm125/reco_p8_ee_Z_Ztautau_ecm125_1.root")

In [ ]:
ev = fi["events"]

In [ ]:
mcp = ev.arrays(["MCParticles"])["MCParticles"]
mcp = awkward.Record({k.replace("MCParticles.", ""): mcp[k] for k in mcp.fields})

In [ ]:
mc_p4 = vector.awk(awkward.zip({
    "mass": mcp["mass"],
    "x": mcp["momentum.x"],
    "y": mcp["momentum.y"],
    "z": mcp["momentum.z"]}))

In [ ]:
mc_p4.energy[(np.abs(mcp["PDG"])==15) & (mcp["generatorStatus"]==2)]

In [ ]:
msk = mcp["generatorStatus"]==1
mc_pid = awkward.flatten(mcp.PDG[msk])
mc_charge = awkward.flatten(mcp.charge[msk])
mc_energy = awkward.flatten(mc_p4.energy[msk])

mc_candid = np.array([
    map_pdgid_to_candid(abs(pdgid), charge) for pdgid, charge in zip(mc_pid, mc_charge)
])

b = np.logspace(-2,3,101)
for pid in np.unique(mc_candid):
    plt.hist(
        mc_energy[mc_candid==pid],
        bins=b,
        histtype="step",
        lw=2,
        label=pid
    );
plt.legend()
plt.xscale("log")
plt.xlabel("Energy [GeV]")
plt.ylabel("Number of particles / bin")
plt.title("Stable gen particles")

In [ ]:
pos = ev.arrays([
    "ECALEndcap/ECALEndcap.position.x",
    "ECALEndcap/ECALEndcap.position.y",
    "ECALEndcap/ECALEndcap.position.z",
    "ECALEndcap/ECALEndcap.energy",
    "ECALBarrel/ECALBarrel.position.x",
    "ECALBarrel/ECALBarrel.position.y",
    "ECALBarrel/ECALBarrel.position.z",
    "ECALBarrel/ECALBarrel.energy",
    
    "HCALEndcap/HCALEndcap.position.x",
    "HCALEndcap/HCALEndcap.position.y",
    "HCALEndcap/HCALEndcap.position.z",
    "HCALEndcap/HCALEndcap.energy",
    "HCALBarrel/HCALBarrel.position.x",
    "HCALBarrel/HCALBarrel.position.y",
    "HCALBarrel/HCALBarrel.position.z",
    "HCALBarrel/HCALBarrel.energy",

])

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(
    pos[0]["ECALEndcap/ECALEndcap.position.x"],
    pos[0]["ECALEndcap/ECALEndcap.position.y"],
    s = pos[0]["ECALEndcap/ECALEndcap.energy"]
)

plt.scatter(
    pos[0]["ECALBarrel/ECALBarrel.position.x"],
    pos[0]["ECALBarrel/ECALBarrel.position.y"],
    s = pos[0]["ECALBarrel/ECALBarrel.energy"]
)

plt.scatter(
    pos[0]["HCALEndcap/HCALEndcap.position.x"],
    pos[0]["HCALEndcap/HCALEndcap.position.y"],
    s = pos[0]["HCALEndcap/HCALEndcap.energy"]
)

plt.scatter(
    pos[0]["HCALBarrel/HCALBarrel.position.x"],
    pos[0]["HCALBarrel/HCALBarrel.position.y"],
    s = pos[0]["HCALBarrel/HCALBarrel.energy"]
)

plt.xlim(-2000,2000)
plt.ylim(-2000,2000)

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(
    pos[0]["ECALEndcap/ECALEndcap.position.x"],
    pos[0]["ECALEndcap/ECALEndcap.position.z"],
    s = pos[0]["ECALEndcap/ECALEndcap.energy"]
)

plt.scatter(
    pos[0]["ECALBarrel/ECALBarrel.position.x"],
    pos[0]["ECALBarrel/ECALBarrel.position.z"],
    s = pos[0]["ECALBarrel/ECALBarrel.energy"]
)

plt.scatter(
    pos[0]["HCALEndcap/HCALEndcap.position.x"],
    pos[0]["HCALEndcap/HCALEndcap.position.z"],
    s = pos[0]["HCALEndcap/HCALEndcap.energy"]
)

plt.scatter(
    pos[0]["HCALBarrel/HCALBarrel.position.x"],
    pos[0]["HCALBarrel/HCALBarrel.position.z"],
    s = pos[0]["HCALBarrel/HCALBarrel.energy"]
)

#plt.xlim(-2000,2000)
#plt.ylim(-2000,2000)